# Sentinel Hub Python Requests

## Setup

#### Imports

In [73]:
from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session
from PIL import Image
import io
import numpy as np
import matplotlib.pyplot as plt

#### Authentication

In [74]:

CLIENT_ID = 'cbbc4a7d-cad7-44b2-92e9-00489b945b62'
CLIENT_SECRET = 'rtdgkVVhPnanYQGXPn8URSZI6V1pafGcTT395yfV'

# set up credentials
client = BackendApplicationClient(client_id=CLIENT_ID)
oauth = OAuth2Session(client=client)

# get an authentication token
token = oauth.fetch_token(token_url='https://services.sentinel-hub.com/oauth/token',
                          client_secret=CLIENT_SECRET, include_client_id=True)

## Request an Image of Barrier Reef

#### Request Parameters

In [75]:
bbox = [47.01962, 28.73135, 46.99954, 28.76276]
start_date = "2023-06-24"
end_date = "2023-06-24"
collection_id = "sentinel-2-l2a"

#### Evalscript and Request Body

In [86]:
# evalscript
evalscript = """
///VERSION=3

function setup() {
  return {
    input: ["B02", "B03", "B04", "B8A", "B11"], // add B8A and B11 bands
    output: { id: 'default',
              bands: 5 } // change bands to 4
  };
}

function evaluatePixel(sample) {
  var ndmi = (sample.B8A - sample.B11) / (sample.B8A + sample.B11); // calculate NDMI
  return [2.5 * sample.B04, 2.5 * sample.B03, 2.5 * sample.B02, ndmi]; // add NDMI to output
}
"""

# request body/payload
json_request = {
    'input': {
        'bounds': {
            'bbox': bbox,
            'properties': {
                'crs': 'http://www.opengis.net/def/crs/OGC/1.3/CRS84'
            }
        },
        'data': [
            {
                'type': 'S2L2A',
                'dataFilter': {
                    'timeRange': {
                        'from': f'{start_date}T00:00:00Z',
                        'to': f'{end_date}T23:59:59Z'
                    },
                    'mosaickingOrder': 'leastCC',
                },
            }
        ]
    },
    'output': {
        'width': 1024,
        'height': 1024,
        'responses': [
            {
                'identifier': 'default',
                'format': {
                    'type': 'image/tiff', # change type to tiff
                    'bands': 'B11,B04,B03,B02,NDMI' # specify bands
                }
            }
        ]
    },
    'evalscript': evalscript
}


#### Send the Request

In [92]:
# Set the request url and headers
url_request = 'https://services.sentinel-hub.com/api/v1/process'
headers_request = {
    "Authorization" : "Bearer %s" %token['access_token']
}

#Send the request
response = oauth.request(
    "POST", url_request, headers=headers_request, json = json_request
)

#print(response.content)

b'MM\x00*\x00\x00\x00\x08\x00\x10\x01\x00\x00\x03\x00\x00\x00\x01\x04\x00\x00\x00\x01\x01\x00\x03\x00\x00\x00\x01\x04\x00\x00\x00\x01\x02\x00\x03\x00\x00\x00\x05\x00\x00\x00\xd0\x01\x03\x00\x03\x00\x00\x00\x01\x80\xb2\x00\x00\x01\x06\x00\x03\x00\x00\x00\x01\x00\x01\x00\x00\x01\x11\x00\x04\x00\x00\x00\x80\x00\x00\x00\xdc\x01\x15\x00\x03\x00\x00\x00\x01\x00\x05\x00\x00\x01\x16\x00\x03\x00\x00\x00\x01\x00\x08\x00\x00\x01\x17\x00\x04\x00\x00\x00\x80\x00\x00\x02\xdc\x01\x1a\x00\x05\x00\x00\x00\x01\x00\x00\x04\xdc\x01\x1b\x00\x05\x00\x00\x00\x01\x00\x00\x04\xe4\x01(\x00\x03\x00\x00\x00\x01\x00\x01\x00\x00\x83\x0e\x00\x0c\x00\x00\x00\x03\x00\x00\x04\xec\x84\x82\x00\x0c\x00\x00\x00\x06\x00\x00\x05\x04\x87\xaf\x00\x03\x00\x00\x00\x1c\x00\x00\x054\x87\xb1\x00\x02\x00\x00\x00\x0f\x00\x00\x05l\x00\x00\x00\x00\x00\x00\x00\x08\x00\x08\x00\x08\x00\x08\x00\x08\x00\x00\x00\x00\x05{\x00\x00\x11\x06\x00\x00\x1c\x80\x00\x00\'\xb0\x00\x002\xed\x00\x00>t\x00\x00I\x99\x00\x00T\xa9\x00\x00_\xfb\x00\x00k\x04\x

#### Visualize the Image

In [90]:
# read the image as numpy array
image_arr = np.array(Image.open(io.BytesIO(response.content)))

# plot the image for visualization
plt.figure(figsize=(16,16))
plt.axis('off')
plt.tight_layout()
plt.imshow(image_arr)

UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x11e19d120>

### Run a Catalog API Request

#### Create the request body/payload

In [ ]:
json_search = {
    'bbox': bbox,
    'datetime': f'{start_date}T00:00:00Z/{end_date}T23:59:59Z',
    'collections': [collection_id],
    'limit': 1
}

#### Set the url and headers for the Catalog API and send a search request

In [ ]:
# set the url and headers
url_search = 'https://services.sentinel-hub.com/api/v1/catalog/search'
headers_search = {
  'Content-Type': 'application/json'
}

# send the request
response_search = oauth.request(
    "POST", url_search, headers=headers_search, json = json_search
)

#### Print the search result

In [ ]:
response_search.json()

{'type': 'FeatureCollection',
 'features': [{'stac_version': '0.9.0',
   'stac_extensions': ['eo', 'projection'],
   'id': 'S2A_MSIL2A_20230624T072621_N0509_R049_T38RPS_20230624T104503',
   'type': 'Feature',
   'geometry': {'type': 'MultiPolygon',
    'crs': {'type': 'name',
     'properties': {'name': 'urn:ogc:def:crs:OGC::CRS84'}},
    'coordinates': [[[[46.02592964844589, 28.92436500128604],
       [46.016416807754766, 27.933406118899885],
       [47.13194567110351, 27.92065311807928],
       [47.151884532018805, 28.91107671330216],
       [46.02592964844589, 28.92436500128604]]]]},
   'bbox': [46.016416807754766,
    27.92065311807928,
    47.151884532018805,
    28.92436500128604],
   'properties': {'datetime': '2023-06-24T07:41:59Z',
    'platform': 'sentinel-2a',
    'instruments': ['msi'],
    'constellation': 'sentinel-2',
    'eo:gsd': 10,
    'eo:cloud_cover': 0.0,
    'eo:bands': [{'name': 'B01',
      'common_name': 'coastal',
      'center_wavelength': 0.4427,
      'ful